1. Thu thập các đường dẫn thuộc mỗi chuyên mục từ trang web https://vnexpress.net/

- Viết hàm lấy tất cả các đường dẫn của bài báo trong trang web

In [1]:
# Gọi thư viện
from urllib.request import urlopen 
from bs4 import BeautifulSoup
import re

# Đường dẫn đến trang web gốc
URL = 'https://vnexpress.net/'

def _get_links(url):
    links = [] 
    html = urlopen(url).read()  
    soup = BeautifulSoup(html, 'html.parser') 

    # Lấy tất cả đường dẫn trong trang web hiện tại
    urls = set(re.findall(r'(?:http|https|ftp):\/\/(?:[\w_-]+(?:\.[\w_-]+)+)(?:[\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', str(soup)))

    for link in urls:
        # Chỉ lấy đường dẫn là trang web nhiều bài báo
        if link != URL and link.startswith(URL) and not link.startswith(URL + 'tac-gia') and link.endswith('.html'):
            links.append(link)

    return links

- Kết quả trả về của hàm với đường dẫn gốc của VNexpress

In [15]:
_get_links(URL)

['https://vnexpress.net/thu-tuong-len-duong-du-hoi-nghi-brics-mo-rong-tai-nga-4807242.html',
 'https://vnexpress.net/minh-anh-nhung-co-gai-chan-dai-toi-song-tuy-duyen-4806910.html',
 'https://vnexpress.net/khong-khi-lanh-tang-cuong-xuong-mien-bac-4807168.html',
 'https://vnexpress.net/tong-cong-to-ukraine-tu-chuc-giua-be-boi-tuyen-quan-4807227.html',
 'https://vnexpress.net/vinicius-lap-hat-trick-real-de-bep-dortmund-4807248.html',
 'https://vnexpress.net/dien-mat-troi-mai-nha-du-thua-duoc-ban-toi-da-20-cong-suat-4807206.html',
 'https://vnexpress.net/kien-nghi-nam-2025-chua-tang-luong-huu-luong-cong-chuc-4806953.html',
 'https://vnexpress.net/mcdonald-s-phu-nhan-moi-trump-den-ran-khoai-tay-4806934.html',
 'https://vnexpress.net/tp-hcm-se-phat-trien-42-cong-vien-ven-song-sai-gon-4807184.html',
 'https://vnexpress.net/horea-ca-nhan-xin-cap-so-do-se-chiu-tac-dong-manh-tu-bang-gia-dat-moi-4807226.html',
 'https://vnexpress.net/nguoi-dan-ong-tu-vong-do-cum-a-h1-n1-4807238.html',
 'https://

- Lấy đường dẫn của các bài toán thuộc từng chuyên mục

In [7]:
# Danh sách các chuyên mục
categories = [
    'thoi-su', 'kinh-doanh', 'khoa-hoc', 'giai-tri',
    'the-thao', 'phap-luat', 'giao-duc', 'suc-khoe',
    'doi-song', 'du-lich', 'so-hoa', 'oto-xe-may'
]

# Danh sách các đường dẫn đến các bài báo của mỗi chuyên mục
article_url = {}

for category in categories:
    article_url[category] = []

    # Lấy 10 trang trong mỗi chuyên mục
    for i in range(1, 11): 
        url = f'{URL}{category}-p{i}'
        article_url[category] += _get_links(url)

    # Loại bỏ đường dẫn trùng nhau
    article_url[category] = list(set(article_url[category]))

    # Hiển thị số đường dẫn thu thập được trong chuyên mục
    print(category, len(article_url[category]))

# Lấy riêng các đường dẫn trong chuyên mục theo góc nhìn từng mục con
goc_nhin = [
    'binh-luan-nhieu', 'chinh-tri-chinh-sach', 'y-te-su-khoe',
    'kinh-doanh-quan-tri', 'giao-duc-tri-thuc', 'moi-truong',
    'van-hoa-loi-song', 'covid-19'
]

article_url['goc-nhin'] = []

for sub_cate in goc_nhin:
    url = f'{URL}/goc-nhin/{sub_cate}'
    article_url['goc-nhin'] += _get_links(url)

# Loại bỏ đường dẫn trùng nhau trong chuyên mục 'goc-nhin'
article_url['goc-nhin'] = list(set(article_url['goc-nhin']))
print('goc-nhin', len(article_url['goc-nhin']))


thoi-su 317
kinh-doanh 320
khoa-hoc 308
giai-tri 311
the-thao 308
phap-luat 314
giao-duc 320
suc-khoe 315
doi-song 314
du-lich 327
so-hoa 313
oto-xe-may 315
goc-nhin 27


2. Lấy nội dung của bài báo ứng với từng đường dẫn cụ thể và lưu vào một file trong folder chuyên mục tương ứng.

- Viết hàm lấy nội dung của bài báo ứng với đường dẫn cụ thể

In [8]:
def _get_content(url):
    # Nội dung bài báo
    content = ''
    
    # Mở URL và đọc nội dung trang
    html = urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    # Lấy phần chứa nội dung bài báo
    div_content = soup.select('.page-detail .container')
    if len(div_content) > 0:
        div_content = div_content[0]

        # Lấy phần mô tả
        description = div_content.find_all('p', {'class': 'description'})
        if len(description) > 0:
            description = description[-1]
            text_description = description.get_text()
            location = description.find('span', {'class': 'location-stamp'})
            if location is not None:
                content = text_description[len(location.get_text()):]
            else:
                content = text_description   

        # Lấy phần chứa nội dung chi tiết bài báo
        detail = div_content.find('article', {'class': 'fck_detail'})
        if detail is not None:
            p_normal = detail.find_all('p', {'class': 'Normal'})
            if len(p_normal) > 0:
                for p in p_normal:
                    p_text = p.get_text()
                    # Kiểm tra đoạn văn bản không rỗng, không bắt đầu bằng dấu '>>' và kết thúc bằng các ký tự hợp lệ
                    if p_text != '' and not p_text.startswith('>>') and p_text[-1] in ['.', '!', '?']:
                        content += ' ' + p.get_text()

    # Xóa ký tự ngắt dòng và trả về
    return re.sub(r'\n', '', content)

+ Kết quả trả về của hàm với đường dẫn dầu tiên trong chuyên mục số hóa

In [20]:
_get_content(article_url['giao-duc'][0])

'Bài học đầu đời về quản lý tài chính giúp trẻ nhận thức về tiền bạc, sử dụng có trách nhiệm và thông minh. Theo các chuyên gia, trước tiên, phụ huynh nên giáo dục trẻ hiểu rằng tiền bạc không tự nhiên có mà phải được tạo ra thông qua công việc hay nỗ lực. Khi trẻ bắt đầu nhận thức được giá trị của tiền, các em sẽ học cách sử dụng đúng đắn. Cha mẹ có thể dạy con bằng cách trả công cho những việc nhỏ, từ đó tạo động lực cho trẻ. Thông qua những bài học đơn giản giúp trẻ biết cách quản lý tài sản, nuôi dưỡng lòng tốt, phát triển bản thân. Dưới đây là một số cách thức giúp cha mẹ giáo dục con về quản lý tài chính. Tiết kiệm là một trong những kỹ năng quan trọng mà trẻ cần học từ sớm. Thói quen này giúp trẻ hiểu giá trị của tiền bạc, rèn luyện tính kiên nhẫn và kỷ luật. Phương pháp thú vị và dễ hiểu là khuyến khích trẻ bỏ tiền vào ống heo mỗi ngày. Thay vì tiêu hết tiền tiêu vặt trong ngày, cha mẹ có thể hướng dẫn trẻ để dành một phần nhỏ và bỏ vào ống heo. Ví dụ, mỗi ngày trẻ có thể bỏ và

- Lưu nội dung các bài báo thành các file nằm trong folder chuyên mục tương ứng

In [9]:
import os

# Danh sách tất cả các câu đã làm sạch của tất cả các bài viết
sentence = []
#Chữ cái in hoa của tiếng việt
uppercase = "AÀÁẢÃẠĂẰẮẲẴẶÂẦẤẨẪẬBCDĐEÈÉẺẼẸÊỀẾỂỄỆFGHIÌÍỈĨỊOÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢPQRSTUÙÚỦŨỤƯỪỨỬỮỰVWXYỲÝỶỸỴZ"

for category in article_url.keys():
    # Tạo folder chuyên mục để chứa các file bài viết
    current_path = 'data/categories/' + category + '/'
    os.mkdir(current_path)

# Lưu các bài viết thành từng file
    count = 0
    for i in range(len(article_url[category])):
        content = _get_content(article_url[category][i])
        if content != '':
            count += 1
            f = open('{}/{}_{}.txt'.format(current_path, category, str(count)), 'w', encoding='utf-8') 
            # Chèn khoảng cách vào sau các dấu câu (., ?, !) nếu không có khoảng trắng hoặc dấu ngoặc kép ngay sau đó
            lines = re.sub(r'(?<=[.?!])(?=\s*[\'"`"’]*[{}])'.format(uppercase), ' ', content)

            # Tách bài viết thành các câu (giả sử câu kết thúc bằng ., !, ? và câu mới bắt đầu bằng chữ cái in hoa hoặc chữ số)
            lines = re.split(r'(?<=[.?!])(?=\s*[{}0-9])'.format(uppercase), content)

            # Ghi từng câu trong vào viết vào file
            for line in lines:
                f.write((line + '\n'))
                sentence.append(line)
            f.close()
        # Sổ bài viết trong chuyên mục    
        print(category, count)

        sentence = list(set(sentence))
        print("Tổng số câu được tách: ", len(sentence))

thoi-su 1
Tổng số câu được tách:  33
thoi-su 2
Tổng số câu được tách:  43
thoi-su 3
Tổng số câu được tách:  56
thoi-su 4
Tổng số câu được tách:  82
thoi-su 5
Tổng số câu được tách:  105
thoi-su 6
Tổng số câu được tách:  119
thoi-su 7
Tổng số câu được tách:  129
thoi-su 8
Tổng số câu được tách:  132
thoi-su 8
Tổng số câu được tách:  132
thoi-su 9
Tổng số câu được tách:  141
thoi-su 10
Tổng số câu được tách:  162
thoi-su 11
Tổng số câu được tách:  187
thoi-su 12
Tổng số câu được tách:  205
thoi-su 13
Tổng số câu được tách:  216
thoi-su 14
Tổng số câu được tách:  233
thoi-su 15
Tổng số câu được tách:  239
thoi-su 16
Tổng số câu được tách:  252
thoi-su 17
Tổng số câu được tách:  262
thoi-su 18
Tổng số câu được tách:  263
thoi-su 19
Tổng số câu được tách:  275
thoi-su 20
Tổng số câu được tách:  285
thoi-su 21
Tổng số câu được tách:  307
thoi-su 22
Tổng số câu được tách:  336
thoi-su 23
Tổng số câu được tách:  348
thoi-su 24
Tổng số câu được tách:  351
thoi-su 25
Tổng số câu được tách:  364


+ 10 câu dầu tiên tách được

In [10]:
sentence[:10]

[' Yagi là cơn bão mạnh nhất 30 năm qua, vùng gần tâm bão mạnh cấp 14, giật cấp 17, ảnh hưởng hầu hết tỉnh thành phía Bắc.',
 " Cả hai mang vàng bán cho 3 tiệm ở thị xã Ayun Pa, tỉnh Gia Lai, được hơn 62 triệu đồng. Khoảng 2h sáng nay, Duyên và Quang bị Công an huyện Ea H'leo bắt giữ, điều tra về hành vi Cướp giật tài sản.",
 ' Bản Sport phân biệt với các bản khác nằm ở trang trí ngoại thất với các chi tiết sơn đen, cốp đóng/mở điện chỉ với một nút bấm. Nội thất tương tự như ngoại thất với các chi tiết trang trí màu đen.',
 'Xe phía trước bật đèn khẩn cấp, đến ngã tư rẽ trái khiến xe phía sau bất ngờ thì có bị phạt. (Nguyễn Sơn) Xin hỏi bật sai tín hiệu như vậy có bị phạt, nếu có mức phạt thế nào?',
 ' Họ vượt đèo Khardung La cao 5.',
 ' Theo thương hiệu, với việc kết hợp công nghệ Ag Clean, sản phẩm còn có khả năng ngăn ngừa tình trạng nhiễm khuẩn chéo, khử mùi hôi trong tủ lạnh, loại bỏ 99,9% nấm mốc, phù hợp với gia đình thường xuyên tích trữ món ăn có mùi mạnh, hoặc đồ ăn sẵn nhiều

<strong>3. Lưu tất cả các câu vào file ngữ liệu </strong>

In [11]:
# File ngữ liệu
file_corpus = open('data/corpus.txt', 'w', encoding='utf-8')

count = 0
for sent in sentence:
    # Xóa ký tự thừa
    sent = re.sub(r'\u200b', '', sent)

    file_corpus.write(sent.strip() + '\n')
    count +=1
file_corpus.close()

print('Số lượng câu trong file ngữ liệu: ', count)

Số lượng câu trong file ngữ liệu:  66710
